In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embeddindg
#import gensim
#from gensim.models import Word2Vec
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
import pickle
with open('./tagged_db.pickle', 'rb') as f:
    dataset = pickle.load(f)
    

In [ ]:
import pandas as pd
df = pd.DataFrame(dataset)
#df.set_index(name, drop=True, append=False, inplace=False, verify_integrity=False)
df = df.transpose()
df['new_col'] = range(1, len(df) + 1)
df = df.set_index("new_col", drop=True, append=False, inplace=False, verify_integrity=False)
df = df.drop(df[df.topic == "filosofia"].index)
df = df.drop(df[df.topic == "geologia"].index)
df = df.drop(df[df.topic == "informatica"].index)
df = df.drop(df[df.topic == "biologia"].index)
df = df.drop(df[df.topic == "nessuno"].index)
df.wordlist[1][2]

In [ ]:
for i, row in df.iterrows():
       row.wordlist =  ' '.join(row.wordlist)

In [ ]:
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('italiano')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [ ]:
def finalpreprocess(string):
    return lemmatizer(preprocess(string))
df['clean_text'] = df['wordlist'].apply(lambda x: finalpreprocess(x))
df.head()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
a = le.fit_transform(df.topic)
df["y"] = a

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"],df["y"],test_size=0.2,shuffle=True)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train_vectors_tfidf, y_train)
y_predict = clf.predict(X_test_vectors_tfidf)
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

In [ ]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 


In [ ]:
X_train_vectors_tfidf.toarray().shape
#y_train.shape


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7585, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64,64)
        self.fc5 = nn.Linear(64,3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x)

net = Net()
print(net)

In [ ]:
X_train = torch.from_numpy(X_train_vectors_tfidf.toarray())
X_test = torch.Tensor(X_test_vectors_tfidf.toarray())
y_train = torch.from_numpy(y_train.values.astype(np.float))
y_test = torch.Tensor((y_test.values.astype(np.float)))
training_dataset = torch.utils.data.TensorDataset(X_train, y_train)
testing_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=256, shuffle=False)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size = 256, shuffle = False)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr= 0.001)

In [ ]:
lo = []
for epoch in range(100):
    for data in train_loader:
        x, y = data
        net.zero_grad()
        x = x.view(-1,7585).float()
        output = net(x)
        loss = loss_function(output, y.type(torch.LongTensor)) # calc and grab the loss value
        loss.backward()
        optimizer.step()
    print(loss)
    lo.append(loss.detach().numpy())

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        X, y = data
        output = net(X.view(-1,7585))
        #print(output)
        for idx, i in enumerate(output):
            #print(torch.argmax(i), y[idx])
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1

print("Accuracy: ", round(correct/total, 3))